In [ ]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, EffectScatter

In [ ]:
df_1d = pd.read_csv("ETH_daily.csv")  

In [ ]:
df_1d["Close_1_shift"] = df_1d["Close"].shift(1)  

In [ ]:
df_1d["TR"] = np.abs(df_1d.High - df_1d.Low)
df_1d["TR"] = np.maximum(
    df_1d["TR"],
    np.maximum(
        np.abs(df_1d.Close_1_shift - df_1d.High),
        np.abs(df_1d.Close_1_shift - df_1d.Low),
    ),
)

In [ ]:
n_array = np.array(df_1d["TR"].values)
n_array[20] = np.mean(df_1d["TR"][:20])
for i in range(21, df_1d.shape[0]):
    n_array[i] = (19.0 * n_array[i - 1] + df_1d["TR"][i]) / 20.0
df_1d["N"] = n_array


In [ ]:
# Compute upper and lower bounds based on Turtle Algorithm
df_1d["upper_bound"] = df_1d["High"].shift(1).rolling(window=20).max()
df_1d["lower_bound"] = df_1d["Low"].shift(1).rolling(window=10).min()
df = df_1d

In [ ]:
crypto_value = [0 for i in range(20)]
money = [1000 for i in range(20)]
init_money = 1000
crypto = 0
fees = 0.001  # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  


In [ ]:
#original turtle, 考虑仓位管理，每次加仓1 unit

for i in range(21, df.shape[0]): 

    if ( crypto == 0 and df["Close_1_shift"].iloc[i] > df["upper_bound"].iloc[i-1] ): #建仓
        # We will use average price from the current ticker
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        unit = 1 *0.01*init_money/df["N"].iloc[i]

        if money[-1] > unit*price*(1+fees):    #剩下的钱够买 1 unit
            crypto += unit
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[money[-1]-unit * price*(1+fees)]

        elif 0<money[-1] and money[-1]<unit*price:     #剩下的钱不够买 1 unit
            crypto += money[-1]*(1.0 - fees)/price
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[0]
            
        increase_position = price + 0.5*df["N"].iloc[i]
        stop_loss = price - 2.0 * df["N"].iloc[i]
        positions += [{ "date": df.Date.iloc[i], "price": price, "money": init_money}]

    elif crypto == 0:    #空仓不动
        crypto_value += [crypto_value[-1]]
        money += [money[-1]]
        
    elif (crypto > 0 and df["Close_1_shift"].iloc[i] > increase_position and money[-1] >0) : #加仓
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        unit = 1 *0.01*init_money/df["N"].iloc[i]
        
        if money[-1] > unit*price*(1+fees):    #剩下的钱够买 1 unit
            crypto += unit
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[money[-1]-unit * price*(1+fees)]

        elif 0<money[-1] and money[-1]<unit*price:     #剩下的钱不够买 1 unit
            crypto += money[-1]*(1.0 - fees)/price
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[0]
            
        increase_position = price + 0.5*df["N"].iloc[i]
        stop_loss = price - 2.0 * df["N"].iloc[i]

        
    elif  crypto > 0 and (
        df["Close_1_shift"].iloc[i]< df["lower_bound"].iloc[i-1] 
        or df["Close_1_shift"].iloc[i] < stop_loss  
        or i == df.shape[0] - 1 ):                      #清仓
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [money[-1] + crypto * price * (1 - fees)]
        init_money = money[-1]
        stop_loss, crypto = 0, 0
        crypto_value += [0]
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price":[positions[-1]["price"],price],
                    "money": [positions[-1]["money"], money[-1]],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price":[positions[-1]["price"],price],
                    "money": [positions[-1]["money"], money[-1]],
                }
            ]
        positions = []
        
    elif crypto > 0:           #持仓不动
        crypto_value += [crypto * df["Close"].iloc[i]]
        money += [money[-1]]


In [ ]:
success_rate = 0
if len(success_history) + len(failure_history) > 0:
    success_rate = len(success_history) / (len(failure_history) + len(success_history))


In [ ]:
print("-" * 50)
print("Benchmark",(df.Close.iloc[df.shape[0]-1]/df.Close.iloc[0]-1)*1000+1000)
print("Capital at the end", np.round(money[-1], 2))
print("Accumulated return at the end", np.round(money[-1]/money[0]*100, 4),"%")
print("Win ratio", success_rate )
print("-" * 50)
print("Summary of % change in positions")
percent_change = []
positions = []
for h in [failure_history, success_history]:
    for position in h:
        percent_change.append(np.round((position["money"][1] - position["money"][0])/ position["money"][0]* 100.0, 2,))
        print("Percent change in position", percent_change[-1])
        positions += [position]
        
print("-" * 50)
print("Total trade times", len(percent_change))
print("Average return per trade", np.round(sum(percent_change)/len(percent_change),2),"%")
print("Max win", max(percent_change),"%")
print("Max loss", min(percent_change),"%")

--------------------------------------------------
Benchmark 23094.746501491165
Capital at the end 5161.87
Accumulated return at the end 516.1873 %
Win ratio 1.0
--------------------------------------------------
Summary of % change in positions
Percent change in position 10.62
Percent change in position -1.13
Percent change in position -2.48
Percent change in position 18.2
Percent change in position 286.39
Percent change in position 0.8
Percent change in position -0.62
Percent change in position 11.78
Percent change in position -1.89
Percent change in position -3.55
--------------------------------------------------
Total trade times 10
Average return per trade 31.81 %
Max win 286.39 %
Max loss -3.55 %
--------------------------------------------------
Benchmark 23094.746501491165
Capital at the end 5161.87
Accumulated return at the end 516.1873 %
Win ratio 1.0
--------------------------------------------------
Summary of % change in positions
Percent change in position 10.62
Percent 

In [ ]:
attr=[str(t) for t in df.Date]
bd = [str(i["date"][0]) for i in positions]
sd = [str(i["date"][1]) for i in positions]
bp = [np.round((i["price"][0]),2) for i in positions]
sp = [np.round((i["price"][1]),2) for i in positions]

v1=np.array(df.loc[:,['Open','Close','Low','High']]).tolist()
v2=np.array(df.upper_bound).tolist()
v3=np.array(df.lower_bound).tolist()
kline1 = (
    Kline()
    .add_xaxis(attr)
    .add_yaxis("ETH",v1)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="original_turtle"),
    ) 
)

line1 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("upper_bound",v2 , is_smooth = True,)
    .add_yaxis("lower_bound",v3 , is_smooth = True,)
)

cb = (
    EffectScatter()
    .add_xaxis(bd)
    .add_yaxis("Buy", bp)
)

cs = (
    EffectScatter()
    .add_xaxis(sd)
    .add_yaxis("Sell", sp)
)

kline1.overlap(line1)
kline1.overlap(cb)
kline1.overlap(cs)
kline1.render("p1_original_turtle.html")


'F:\\jupyter\\p1_original_turtle.html'

'F:\\jupyter\\p1_original_turtle.html'

In [ ]:
rtn = [np.round((crypto_value[i]+money[i])/money[0],4) for i in range(df.shape[0]-1)]
ben = [np.round(j/df.Close.iloc[0],4) for j in df.Close]

line2 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("rtn",rtn , is_smooth = True,)
    .add_yaxis("ben",ben , is_smooth = True,) 
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="original_turtle"),
    ) 
)

line2.render("p2_original_turtle.html")


'F:\\jupyter\\p2_original_turtle.html'

'F:\\jupyter\\p2_original_turtle.html'